# course  details

In [1]:

course = 4

#online (1) or offline (0)
connection = 0

# for online connections you have the option to download all course data to the computer.
# download(1) skip_download(0)
download_csv = 0

# Times should be in US/East local time
class_start = '2021-04-20'
class_end   = '2021-04-22'

#input your student device name
device_name = 'Guthrum'

#status
print('Global Variables Set')

Global Variables Set


In [2]:
%%bash
pip install pandas
pip install boto3
pip install geopandas
pip install keplergl
pip install networkx

# configuration setup for the notebook libraries and Methods

In [3]:
%%bash
pip install scipy
pip install matplotlib
pip install h3

In [4]:
#import libraries, functions, methods, configurations
%run studentConfig.py

#status
print('Config Successful')

Module Import Completed
Config Successful


In [5]:
print("offline notebook use. Reading data from local .csv file....")
filePath = "appIOTcourse_4_allStudent_data.csv.csv"

all_classData = pd.read_csv(f"{filePath}", low_memory=False)

#inspect the schema
all_classData.info()

offline notebook use. Reading data from local .csv file....
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200085 entries, 0 to 200084
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  200085 non-null  int64  
 1   dtg                 200085 non-null  object 
 2   name                200085 non-null  object 
 3   uuid                200085 non-null  object 
 4   source              200085 non-null  object 
 5   lat                 200085 non-null  float64
 6   lon                 200085 non-null  float64
 7   unix_timestamp      200085 non-null  int64  
 8   utc_ts              200085 non-null  object 
 9   localized_ts        200085 non-null  object 
 10  tzID                200085 non-null  object 
 11  platform_country    0 non-null       float64
 12  platform_bandwidth  0 non-null       float64
 13  accuracy            200085 non-null  float64
 14  gps_speed           2000

# read your data into a pandas dataframe

## format for time

In [ ]:
#format date columns
all_classData['utc_ts'] = all_classData['utc_ts'].astype(str)
all_classData['utc_ts'] =  pd.to_datetime(all_classData['utc_ts'].str.strip(), errors='coerce', utc = True)
#add time zone id
all_classData['tzID'] = 'US/Eastern'
all_classData['localized_ts'] = all_classData['utc_ts'].dt.tz_convert('US/Eastern')
all_classData['dtg'] = all_classData['localized_ts'].dt.date
all_classData['dtg'] = pd.to_datetime(all_classData['dtg'])
all_classData.info()

In [6]:
# format dataframe for plotting
plotAll = all_classData.copy(deep=True)

# plot the data
map_1 = KeplerGl(height=525)
map_1.add_data(data=plotAll, name='all_data')
map_1.config = config1
map_1.save_to_html(data={"plotAll": df}, config=config1, file_name='second_map.html')
#map_1.save_to_html(file_name='first_map.html')
#html_string = map_1._repr_html_()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to first_map.html!


In [9]:
from IPython.core.display import display, HTML
from IPython.display import IFrame

#h = f"<iframe srcdoc='{html_string}' width=700 height=600 ></iframe>"
#display(HTML(h))


IFrame(src='./first_map.html', width=700, height=600)

# inspect your dataframe with Kepler.go

# Query for your device mobile ID by the device's name

In [ ]:
# lookup the mid based upon the name of the student's device
#check if online or offline mode
if connection == 1:
    #online
    mid = get_mid(device_name,course)
    print("Student mid (online):")
    print(mid)
elif connection == 0:
    #offline
    mask = offline_lookup['name'] == device_name
    row = offline_lookup.loc[mask]
    mid = row.iloc[0]['udid']
    print("Student mid (offline):")
    print(mid)
else:
    #user entry error
    print("connection selection is invalid")

# Query for your data by the mobile ID (UUID)

In [ ]:
#display student device data by the mid associated with the device global varible
# The results should be the same as the previous command
mask = all_classData['uuid'] == mid
yourData = all_classData.loc[mask]

print(f"Your devices mid is: {mid}")
yourData.head(3)

# Let's look at the IoT details captured for you device

In [ ]:
#display student device IoT by the device_name global varible
mask = yourData['source'] == 'iot'
yourIOT = yourData.loc[mask]
yourIOT = yourIOT.reset_index()
print(f"IOT Collections on device: {device_name}") 
yourIOT.head(3)

# Let's Look at the APP data

In [ ]:
#display student device IoT by the device_name global varible
mask = yourData['source'] == 'app'
yourAPP = yourData.loc[mask]
yourAPP = yourAPP.reset_index()
yourAPP = yourAPP.iloc[:,0:13]
print(f"First party APP collections on device: {device_name}") 
yourAPP.head(3)

## Plot your app data

In [ ]:
# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=yourAPP, name='yourApp_data')
map_2.config = config_yourData
map_2

# filter your data by velocity

In [ ]:
# get dataframe to work on
df = yourAPP.copy(deep=True)

#add a unique ID to dataframe for testing or joining with other dataframes
df.insert(0, 'num', range(0, 0 + len(df)))

#work with time to get the tdoa value in sec and convert it to hours
df['time_delta'] = df.sort_values(['name','utc_ts']).groupby('name')['utc_ts'].diff()
df['time_delta_seconds'] = (df['time_delta'].dt.total_seconds())
df['time_delta_seconds'] = df['time_delta_seconds'].fillna(0)
df['time_delta_hours'] = df['time_delta_seconds'] / 3600
# make the first point (start point of the journey) 0 since it will be a nan
df['time_delta_hours'] = df['time_delta_hours'].fillna(0)
#clean up NaT because kepler does not like NaT when you plot
df.drop('time_delta', axis=1, inplace=True)

# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes (float)

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

#shift df and add a coloumn for the original value of lat and lon
df['orig_lat'] = df.groupby(['name'])['lat'].transform(lambda x:x.shift())
df['orig_lon'] = df.groupby(['name'])['lon'].transform(lambda x:x.shift())
df['dist_km'] = haversine(df['lat'], df['lon'], df['orig_lat'], df['orig_lon'])
# make the first point (start point of the journey) 0 since it will be a NAT
df['dist_km'] = df['dist_km'].fillna(0)
df['dist_miles']=df['dist_km']/1.609
#obtain the velocity
df['velocity_kmph'] = (df['dist_km'] / df['time_delta_hours']).fillna(0)
df['velocity_mph'] = (df['dist_miles'] / df['time_delta_hours']).fillna(0)
#clean temp columns for lat / lon orininals
#df.drop('orig_lat', axis=1, inplace=True)
#df.drop('orig_lon', axis=1, inplace=True)

filter = df[(df.velocity_mph <11)]                             #<------------------------Velocity limit goes here (11)
map_1 = KeplerGl(height=525)
map_1.add_data(data=filter, name='all_data')
map_1.config = config1
map_1

In [ ]:
#filter out your velocity over a certain velocity range
filter = df[(df.velocity_mph >35) & (df.velocity_mph <65)]

# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=filter, name='filter_mph')
map_2.config = config_yourData
map_2

# Inspect your data by source type

In [ ]:
# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=yourAPP, name='app_data')
map_2.add_data(data=yourIOT, name='IOT_data')
map_2.config = config_yourData
map_2

# Section 2 | Data Reduction and Visualization Based on DataTime

In [ ]:
#Set DateTime Range to Align with Exercise or Short Time Window of Interest
time1 = '2021-04-20 08:05:49+00:00'
time2 = '2021-04-20 08:06:51+00:00'

#Get Dataframoe to work on and set ts column to utc time
plotWindow = yourData.copy(deep=True)

#Filter the DateTime Range
mask = (plotWindow['localized_ts'] > time1) & (plotWindow['localized_ts'] < time2) 
#Run query
filtered_data = plotWindow.loc[mask]
filtered_data.head(10)

In [ ]:
# plot the data
map_2 = KeplerGl(height=525)
map_2.add_data(data=filtered_data, name='time_filtered_data')
map_2.config = config2
map_2

# Section 3 | Multi-Device Dataset Import, and Visualize

In [ ]:
#Get Dataframoe to work on and obtain all the student name and the number of events observed by each one
allData = all_classData.copy(deep=True)
allData = allData.groupby(['name','uuid']).size().reset_index().rename(columns={0:'count'})
#display the results
allData

In [ ]:
udid1 = 'bea8836d-3008-44a3-8c3c-53555f0a2dee'
udid2 = '32bdb1af-fd87-4096-a683-63cd16a264d6'

#make a local copy of the date to use for plotting
allData1 = all_classData.copy(deep=True)

#select the udid's we want to look at
allData1 = allData1[allData1['uuid'].isin([udid1,udid2])]
allData1['name'].unique()

In [ ]:
# plot the data
map_3 = KeplerGl(height=525)
map_3.add_data(data=allData1, name='two_devices')
map_3.config = config3
map_3

# Section 4 | Uber H3 HexBins - Associated Locations by Visit Geo [Where Who Where]

#### Geospatial indexing is essential for aggregating and querying data at scale. This type of data is generally abundant, difficult to index or search, and can be structurally complex. Polygons that discriminate particular areas can be very complex to handle. Think of a city’s boundaries. How many points do you need to efficiently determine if a given vehicle has entered a town or even a gas station? The more points you need, the more calculations you will require from your CPU. Overburdening your hardware translates to slower response times and higher resource usage.
#### An efficient geospatial indexing system helps you overcome these hurdles. In the case of H3, the solution takes the form of a hashing scheme.

#### The H3 algorithm partitions the Earth’s surface into a network of hexagons. You can select the amount of detail each hexagon contains by choosing among the available sixteen levels.

In [ ]:
# display the resolution available for h3 indexing with hexagons
hexagon_table = h3_indexTable()
hexagon_table

### To put these areas into context see the below descriptions:
#### *Resolution 7: Roughly city-sized
#### *Resolution 8: Roughly neighborhood-sized
#### *Resolution 9: Roughly 2-4 city blocks in size (different cities have different block sizes)
#### *Resolution 10: Less than a city block in size
#### *Resolution 15: Coffee table-sized

### Now let's perform a fast co-location approach based upon more than one device in a H3 hexagon at res = 9

In [ ]:
# Keep the resolution at 9; the higher the number, the more granular the hexbins become
resolution = 9
#this would meen any device that colocalates with any other device in a hexagon more than one time ( > 1 = 2 devices) 
colo_criteria = 1 

#perform the colocation analysis by h3 index binning
hex_colo = colocate_byH3(all_classData,resolution,colo_criteria)

#Display Partial Data Set  
hex_colo.loc[0:1,:]

In [ ]:
# plot the data
hexagons = pd.DataFrame(hex_colo.hexID)
colo_devices = pd.DataFrame(hex_colo[['hexID','uuid','lat','lon','num_unique','activity_dates_of_UUIDs']])

map_4 = KeplerGl(height=525)
map_4.add_data(data=colo_devices, name='colo_devices')
map_4.add_data(data=hexagons, name='hex_res9')
map_4

# Section 5 | Graphical Visualizatin Comparing UUIDs and SSIDs [Technological Proximity]

In [ ]:
Need a graph example for people being connected together in time and space